# LELA60331 Week 8 Seminar Workbook

> Add blockquote



This week we will look at...

NOTE: THIS SHEET IS JUST A CODE DUMP AT THE MOMENT. IT WILL BE TURNED INTO A STUDENT NOTEBOOK SOON.

Logistic regression with 2 predictors

In [14]:
import numpy as np
## Create simulated data
w1_center = (3, 3)
w2_center = (3, -2)
batch_size=50

x = np.zeros((batch_size, 2))
ground_truth = np.zeros(batch_size)
for i in range(batch_size):
    if np.random.random() > 0.5:
        x[i] = np.random.normal(loc=w1_center)
    else:
        x[i] = np.random.normal(loc=w2_center)
        y[i] = 1

In [144]:
num_features=2
num_hidden_nodes=2
weights_output = np.random.rand(num_hidden_nodes)
weights_hidden = np.random.randn(num_features,num_features)
bias_hidden=np.zeros((1, num_hidden_nodes))
bias_out=np.zeros((1, num_hidden_nodes))

In [147]:
import math
n_iters = 2000
num_features = 2
num_samples = len(y)
lr=0.4

#def relu(X):
#	return np.maximum(X, 0)

def relu_derivative(X):
	return 1. * (X > 0)

for i in range(n_iters):

    z1=x.dot(weights_hidden)+bias_hidden
    #print("ONE")
    #print(z1)
    q1= np.maximum(z1, 0)
    #print("TWO")
    #print(q1)
    z2=q1.dot(weights_output)+np.repeat(np.sum(bias_out,axis=1),50)
    #print(np.repeat(sum(bias_out),50).shape)
    #print(z2)
    #print(bias_out)
    #+bias_out
    out = 1 / (1 + pow(math.e,-z2))
    # Calculating the loss
    loss = sum(-(y*np.log2(out)+(1-y)*np.log2(1-out)))
    print(loss)


    #print(out.dot(weights_output.T))
    #delta2 = out.dot(weights_output.T) * relu_derivative(a1) #if ReLU
    delta2 = np.array([out.dot(weights_output[0])*relu_derivative(q1[:,0]),out.dot(weights_output[1])*relu_derivative(q1[:,1])])
    dw1 = np.dot(x.T, delta2.T)
    db1 = np.sum(delta2.T,axis=0)
    dw2 = (q1.T).dot(out)
    db2 = np.sum(out, axis=0, keepdims=True)
    weights_hidden -= lr * dw1
    bias_hidden -= lr * db1
    weights_output -= lr * dw2
    bias_out -= lr * db2





687.5893248187333
687.6292578091329
687.6691647353249
687.7090456312836
687.748900530921
687.7887294680819
687.8285324765445
687.8683095900219
687.9080608421613
687.947786266543
687.9874858966838
688.0271597660352
688.0668079079824
688.106430355848
688.1460271428863
688.1855983022901
688.2251438671881
688.2646638706416
688.3041583456511
688.3436273251529
688.3830708420168
688.422488929053
688.4618816190057
688.501248944557
688.5405909383248
688.5799076328659
688.6191990606735
688.6584652541767
688.6977062457461
688.7369220676857
688.7761127522401
688.8152783315932
688.8544188378625
688.8935343031073
688.9326247593259
688.9716902384536
689.010730772365
689.0497463928745
689.0887371317353
689.1277030206388
689.1666440912196
689.205560375045
689.2444519036305
689.283318708426
689.3221608208215
689.3609782721514
689.3997710936844
689.4385393166372
689.4772829721597
689.5160020913484
689.5546967052387
689.5933668448058
689.6320125409678
689.6706338245837
689.7092307264555
689.7478032773254


In [255]:
import math
n_iters =5000
input_size = 2
hidden_size = 2
output_size = 1
num_samples = len(y)
learning_rate= 0.1

weights_input_hidden = np.random.randn(input_size, hidden_size)
weights_hidden_output = np.random.randn(hidden_size, output_size)
bias_hidden = np.zeros((1, hidden_size))
bias_output = np.zeros((1, output_size))




def sigmoid(x):
    return 1 / (1 + np.exp(-x))

for i in range(n_iters):

    hidden_input = np.dot(x, weights_input_hidden) + bias_hidden
    hidden_output = sigmoid(hidden_input)
    final_input = np.dot(hidden_output, weights_hidden_output) + bias_output
    final_output = sigmoid(final_input)
    output_error = final_output.squeeze() - y
    #hidden_error = np.dot(output_error, weights_hidden_output.T) * hidden_output * (1 - hidden_output)
    hidden_error = np.sum(output_error*weights_hidden_output,axis=1) * hidden_output * (1 - hidden_output)
    #loss = -np.sum(y * np.log(final_output)) / x.shape[0]
    loss = np.sum(-(y*np.log2(final_output)+(1-y)*np.log2(1-final_output)))
    print("LOSS:")
    print(loss)
    y_pred=[int(ql > 0.5) for ql in final_output]
    acc=[int(yp == y[s]) for s,yp in enumerate(y_pred)]
    print(sum(acc)/len(acc))

    weights_hidden_output -= [[learning_rate * np.dot(hidden_output.T, output_error)[0]],[learning_rate * np.dot(hidden_output.T, output_error)[1]]]
    bias_output -= learning_rate * np.sum(output_error, axis=0, keepdims=True)
    weights_input_hidden -= learning_rate * np.dot(x.T, hidden_error)
    bias_hidden -= learning_rate * np.sum(hidden_error, axis=0, keepdims=True)


LOSS:
2508.7054960721503
0.6
LOSS:
1843.266095383006
0.82
LOSS:
1714.6932471598493
0.82
LOSS:
1762.6018673560611
0.82
LOSS:
1788.9702885079691
0.82
LOSS:
1836.2939700078573
0.82
LOSS:
1879.215020369686
0.82
LOSS:
1919.095763088591
0.82
LOSS:
1956.00353855424
0.82
LOSS:
1989.7539500827593
0.82
LOSS:
2020.7754714881332
0.82
LOSS:
2050.1420423976306
0.82
LOSS:
2078.395570987977
0.82
LOSS:
2105.3375167559416
0.82
LOSS:
2130.8152543870465
0.82
LOSS:
2154.842677766833
0.82
LOSS:
2177.5039909848692
0.82
LOSS:
2198.9069725329073
0.82
LOSS:
2219.1623676871086
0.82
LOSS:
2238.375011663945
0.82
LOSS:
2256.6405075145813
0.82
LOSS:
2274.0444886394143
0.82
LOSS:
2290.663026681797
0.82
LOSS:
2306.563491761616
0.82
LOSS:
2321.8055375751346
0.82
LOSS:
2336.442063092002
0.82
LOSS:
2350.5200899505653
0.82
LOSS:
2364.0815365433446
0.82
LOSS:
2377.1638891146067
0.82
LOSS:
2389.800778382028
0.82
LOSS:
2402.0224729686374
0.82
LOSS:
2413.8563011638234
0.82
LOSS:
2425.327011659918
0.82
LOSS:
2436.4570826413237

<ipython-input-255-e2546994e581>:33: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred=[int(ql > 0.5) for ql in final_output]


Streaming output truncated to the last 5000 lines.
6060.70665905586
0.82
LOSS:
6061.397670340251
0.82
LOSS:
6062.088612529309
0.82
LOSS:
6062.779485660493
0.82
LOSS:
6063.470289657934
0.82
LOSS:
6064.161024502735
0.82
LOSS:
6064.851690419042
0.82
LOSS:
6065.542287207583
0.82
LOSS:
6066.232815066692
0.82
LOSS:
6066.923273926361
0.82
LOSS:
6067.613663747456
0.82
LOSS:
6068.303984632295
0.82
LOSS:
6068.994236603625
0.82
LOSS:
6069.684419722944
0.82
LOSS:
6070.374533792967
0.82
LOSS:
6071.064579181999
0.82
LOSS:
6071.754555616835
0.82
LOSS:
6072.444463154193
0.82
LOSS:
6073.134302020454
0.82
LOSS:
6073.824072039828
0.82
LOSS:
6074.513773331146
0.82
LOSS:
6075.203405902179
0.82
LOSS:
6075.892969690142
0.82
LOSS:
6076.582464771271
0.82
LOSS:
6077.271891200398
0.82
LOSS:
6077.961248865757
0.82
LOSS:
6078.6505380524995
0.82
LOSS:
6079.339758474582
0.82
LOSS:
6080.028910409415
0.82
LOSS:
6080.71799365453
0.82
LOSS:
6081.40700838234
0.82
LOSS:
6082.095954605426
0.82
LOSS:
6082.784832245794
0.82
